In [130]:
import pandas as pd
import nltk

df=pd.read_csv('Data.csv', encoding = "ISO-8859-1")


#the above dataframe has 25 column of top news for everyday in a specific time period 2000-2016
df.head()


,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,Derby raise a glass to Strupar's debut double,"Southgate strikes, Leeds pay the penalty",Hammers hand Robson a youthful lesson,Saints party like it's 1999,Wear wolves have turned into lambs,Stump mike catches testy Gough's taunt,Langer escapes to hit 167,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,Hopkins 'furious' at Foster's lack of Hannibal...,Has Cubie killed fees?,A tale of two tails,I say what I like and I like what I say,"Elbows, Eyes and Nipples",Task force to assess risk of asteroid collision,How I found myself at last,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2,2000-01-05,0,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,McGrath puts India out of their misery,Blair Witch bandwagon rolls on,Pele turns up heat on Ferguson,Party divided over Kohl slush fund scandal,Manchester United (England),Women in record South Pole walk,Vasco da Gama (Brazil),South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,2000-01-06,1,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,England's decade of disasters,Revenge is sweet for jubilant Cronje,"Our choice, not theirs",Profile of former US Nazi Party officer Willia...,New evidence shows record of war crimes suspec...,The rise of the supernerds,Written on the body,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
4,2000-01-07,1,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,Alan Parker - part two,"Thuggery, Toxins and Ties",Met faces fresh attack on race crime,Everton fans top racist 'league of shame',"Our breasts, ourselves",Russia's new boss has an extremely strange his...,Always and forever,Most everywhere: UDIs,Most wanted: Chloe lunettes,

In [131]:
df.tail()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
4096,2016-06-27,0,Barclays and RBS shares suspended from trading...,Pope says Church should ask forgiveness from g...,Poland 'shocked' by xenophobic abuse of Poles ...,"There will be no second referendum, cabinet ag...","Scotland welcome to join EU, Merkel ally says",Sterling dips below Friday's 31-year low amid ...,No negative news about South African President...,Surge in Hate Crimes in the U.K. Following U.K...,Weapons shipped into Jordan by the CIA and Sau...,Angela Merkel said the U.K. must file exit pap...,In a birth offering hope to a threatened speci...,Sky News Journalist Left Speechless As Leave M...,Giant panda in Macau gives birth to twins,Get out now: EU leader tells Britain it must i...,Sea turtle 'beaten and left for dead' on beach...,German lawyers to probe Erdogan over alleged w...,"Boris Johnson says the UK will continue to ""in...",Richard Branson is calling on the UK governmen...,Turkey 'sorry for downing Russian jet',Edward Snowden lawyer vows new push for pardon...,Brexit opinion poll reveals majority don't wan...,"Conservative MP Leave Campaigner: ""The leave c...","Economists predict UK recession, further weake...","New EU 'superstate plan by France, Germany: Cr...",Pakistani clerics declare transgender marriage...
4097,2016-06-28,1,"2,500 Scientists To Australia: If You Want To ...","The personal details of 112,000 French police ...",S&amp;P cuts United Kingdom sovereign credit r...,Huge helium deposit found in Africa,CEO of the South African state broadcaster qui...,"Brexit cost investors $2 trillion, the worst o...",Hong Kong democracy activists call for return ...,Brexit: Iceland president says UK can join 'tr...,UK's Osborne: 'Absolutely' going to have to cu...,'Do not let Scotland down now' : Scottish MEP ...,British pound could hit history-making dollar ...,"Merkel vows to strengthen EU, tells UK no 'che...","""Ryanair will not deploy new aircraft on route...","People, ever more greedy and stupid, destroy t...",Siemens freezes new UK wind power investment f...,"US, Canada and Mexico pledge 50% of power from...",There is increasing evidence that Australia is...,"Richard Branson, the founder of Virgin Group, ...","37,000-yr-old skull from Borneo reveals surpri...",Palestinians stone Western Wall worshipers; po...,Jean-Claude Juncker asks Farage: Why are you h...,"""Romanians for Remainians"" offering a new home...",Brexit: Gibraltar in talks with Scotland to st...,8 Suicide Bombers Strike Lebanon,Mexico's security forces routinely use 'sexual...
4098,2016-06-29,1,Explosion At Airport In Istanbul,Yemeni former president: Terrorism is the offs...,UK must accept freedom of movement to access E...,Devastated: scientists too late to captive bre...,British Labor Party leader Jeremy Corbyn loses...,A Muslim Shop in the UK Was Just Firebombed Wh...,Mexican Authorities Sexually Torture Women in ...,UK shares and pound continue to recover,Iceland historian Johannesson wins presidentia...,99-Million-Yr-Old Bird Wings Found Encased in ...,A chatbot programmed by a British teenager has...,The Philippine president-elect said Monday he ...,Former Belgian Prime Minister ridicules Nigel ...,Brexiteer Nigel Farage To EU: 'You're Not Laug...,Islamic State bombings in southern Yemen kill ...,"Escape Tunnel, Dug by Hand, Is Found at Holoca...",The land under Beijing is sinking by as much a...,Car bomb and Anti-Islamic attack on Mosque in ...,Emaciated lions in Taiz Zoo are trapped in blo...,Rupert Murdoch describes Brexit as 'wonderful'...,More than 40 killed in Yemen suicide attacks,Google Found Disastrous Symantec and Norton Vu...,Extremist violence on the rise in Germany: Dom...,BBC News: Labour MPs pass Corbyn no-confidence...,Tiny New Zealand town with 'too many jobs' lau...
4099,2016-06-30,1,Jamaica proposes marijuana dispensers for tour...,Stephen Hawking says p

In [132]:
#cleaning and preparing the data
#the first column is date , the second is label if the stock price increase or the opposite
#so we can remove the date column as we dont need it for our model, but befor that we can use it to split our data frame for test and train.

#df['Date'] = pd.to_datetime(df['Date'])
#split_date = '20130101'
#train_df = df[df['Date'] < split_date]
#test_df = df[df['Date'] >= split_date]
#print(df['Date'])

train_df = df[df['Date'] < '20150101']
test_df = df[df['Date'] >= '20150101']

train_df.tail()


,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
3970,2015-12-24,0,"Brazil declares emergency after 2,400 babies a...",Scientists find big 1500-yr-old Viking settlem...,Paris attacks: Belgian police arrest ninth sus...,Wiretapping reveals communication between Turk...,Russia airstrikes: Hundreds of Syrian civilans...,Pakistan changes neutral position on Syrian Ci...,North Korean diplomat arrested in South Africa...,German teachers want 'Mein Kampf' on syllabus,North Korea slave force earns Kim Jong Un regi...,"50,000 Yemeni Forces Preparing to Attack Saudi...",Irish government rules out fracking in the Rep...,Americans in Beijing warned of terrorism threa...,"Shin Bet arrests Nazareth area cousins, ISIS o...",Baghdad holds Christmas celebrations out of re...,French cabinet backs plans to embed emergency ...,Report on sexual exploitation and abuse by pea...,Baby girl dies after X-president's son's staff...,China smog sparks red alerts in 10 cities,US planned East Berlin's 'systematic destructi...,Qatar World Cup 2022 workers 'earning as littl...,Thai Court Sentences Two to Death for Killings...,Germany Pays to Halt Danish Wind Power to Prot...,Egypts President Sisi Urges Islamic Scholars t...,Russia Rejects Amnesty Internationals Report o...,"Cameron, Osborne and Murdoch back together at ..."
3971,2015-12-28,0,"Germany recruits 8,500 teachers to teach Germa...","Almost 23,000 people have signed an Italian po...",Japan apologizes to Korea over sexual enslavem...,Milan and Rome ban cars as pollution rises. Mi...,Saudi posts record $98 bn deficit in 2015,"The United States, Russia, Iran, and Saudi Ara...",Vladimir Putin has achieved his central goal o...,Noted Syrian journalist who uncovered ISIS Ale...,Criminal gangs are contaminating democracy in ...,ISIS leader threatens to strike Israel,Schools do not have to teach non-religious wor...,Iraqi army declares first major victory over I...,Syrian journalist &amp; filmmaker who exposed ...,"Train carrying 200,000 litres (44,000 gallons)...","Stoner Sloth was $500,000 well spent, insists ...",Saudi Arabia Plans Subsidy Cuts as King Unveil...,Syria anti-ISIS documentary maker 'assassinate...,Denmark wants Geneva Convention debate if Euro...,David Cameron is heckled during visit to flood...,British Army Is Deployed as Flooding Submerges...,Brazils reluctance to accept an Israeli ambass...,Russian GDP down 4.0 percent year-on-year in N...,Syria and Iraq: Ethnic cleansing by Sunni and ...,Deadly car bomb explodes at Kabul airport,Experiment in Germany tests the idea of an 'em...
3972,2015-12-29,1,Pentagon thwarts Obama's effort to close Guant...,Rare Footage of Giant Squid in Japan,Muslims tell ISIS they'd rather see Star Wars ...,Syrian Journalist Who Documented ISIS Atrociti...,North Korea's second most-powerful man sent to...,Austria Turns Away Hundreds of Migrants for Ly...,Saudi Grand Mufti calls ISIS part of the Israe...,"'What has war yielded, neither land nor heaven...",Ancient arch attacked by ISIS to be rebuilt in...,Islamic State theologians have issued an extre...,A new type of basaltic rock has been discovere...,Belgians arrest 2 accused in New Year's terror...,Germany gives green light to bicycle highways,"""Chinese companies that helped fuel the global...",Five years in jail for men who 'emotionally bu...,Iran Hands Over Stockpile of Enriched Uranium ...,Registrar Shuts Down All Pirate Bay Domain Names,The Irish Government has taken unprecedented a...,Saudi Arabia to raise domestic petrol prices b...,Saudi Arabia hikes petrol prices by 40% at the...,World Health Organization declares Guinea Ebol...,Hacker group Anonymous is helping the 'Making ...,Russia 'has removed uranium from Iran',China threatens to shoot down Australian aircr...,Israel warns Brazil to accept its settler amba...
3973,2015-12-30,0,Tech companies face criminal charges if they n...,Mexican marijuana fa

In [133]:
test_df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
3975,2016-01-04,0,Saudi Arabia breaks off ties with Iran - Amid ...,Exclusive: UK Government urged to reveal its r...,China halts trading as stocks fall 7%,Earthquake measuring 6.8 magnitude strikes India,A virus linked to shrinking newborns brains is...,"New photos of SpaceX booster show sooty, but u...",Indian consulate in Afghanistan under attack b...,Iran calls off Major Hajj,Denmark responds to Swedish border checks with...,David Cameron criticised for turning 'blind ey...,Denmark introduces temporary controls along Ge...,Israel Retaliates to Hezbollah bombing at Leba...,China Aims to Build 40 Nuclear Power Plants in...,"According to obtained documents, after rape an...",Western Australian rooftop solar systems produ...,Iranian leaders website posts controversial im...,"After Bahrain, Sudan expels Iranian ambassador...",Saudi Arabia to cut off all air traffic with I...,Hezbollah sets off large border bomb targeting...,Iranian diplomats given 48 hours to leave Saud...,BBC News: Migrant crisis: Sweden border checks...,Teachers claim the politics of austerity is le...,The Aftermath of a Muslim Cleric's Execution i...,Iran: 40 arrested for attack on Saudi embassy,"Oil prices jump over 2 pct on Saudi Arabia, Ir..."
3976,2016-01-05,1,"Dutch government backs strong encryption, cond...",ISIS Militants Strap Bomb on 4 Year-Old Boy an...,Canada proceeding with controversial $15-billi...,Brazil is heading for its worst recession sinc...,British ISIS militant's desperate plea for doc...,Malaysia's schools will start holding anti-ISI...,Saudi Arabia omitted from UK death penalty str...,ISIS executes first female journalist. One of ...,The U.S. Justice Department has sued Volkswage...,Leaked memo shows Saudi Arabia was fully prepa...,Sister of British extremist fighter says she w...,Mexico first country to approve Israeli drug t...,Jihadi Junior - 6 or 7 year old boy in new Isl...,Nuns claim they were forced to brand themselve...,Malaysia Airlines temporarily bans checked bag...,'Fat cat Tuesday' as top bosses pay overtakes ...,"""The United Nations special rapporteur for hum...",Merkel rejects Bavarian bid for refugee limit:...,Yemen centre for blind 'hit in Saudi coalition...,Mass murderer Anders Behring Breiviks court ca...,Kuwait has recalled its ambassador to Iran in ...,Britain set for first mass strike by doctors i...,First known hacker-caused power outage signals...,Kuwait becomes latest Saudi ally to downgrade ...,'Do Your Cars Emit Oxygen?' Indian Supreme Cou...
3977,2016-01-06,0,China shuts down stock market after 15 min 7% ...,Armed suspect shot dead after trying to storm ...,Oil price falls below $35 a barrel to fresh 11...,Germans outraged by mayors advice for women af...,German public broadcaster ZDF has apologised f...,Iran accuses Saudis of bombing their embassy i...,Protesters demand Canada cancel $15B Saudi con...,"'Snooper's charter' will cost British lives, M...",Islamic scholars from Malaysia and Indonesia i...,Turkish workshop employs Syrian minors to prod...,"100 murdered Christians who were gang-raped, d...",South Korea to restart propaganda loudspeakers...,Egypt asks Israel to keep Turkey away from Gaza,Reducing sugar content in sugar-sweetened drin...,Scientists Discover Meteorite 'Older Than Eart...,Rio Olympic stadium has no water or electricit...,Saudi-led air strikes in Yemen hit centre for ...,Truck bomb kills 65 at Libyan police training ...,Israeli Tourists on Tour Bus Attacked by Maske...,Israel and China have signed a document of und...,Donald Trump threatens to pull 700m investment...,Israel busts Hamas terror cell planning abduct...,A working hospital in central China was demoli...,Paris to close Champs Elysees to cars one Sund...,TransCanada Corp files US$15-billion lawsuit a...
3978,2016-01-07,0,Reports of sexual assaults on women across Eur...,S

In [134]:
data = train_df.iloc[:,2:27]
data.head()

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,Derby raise a glass to Strupar's debut double,"Southgate strikes, Leeds pay the penalty",Hammers hand Robson a youthful lesson,Saints party like it's 1999,Wear wolves have turned into lambs,Stump mike catches testy Gough's taunt,Langer escapes to hit 167,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,Hopkins 'furious' at Foster's lack of Hannibal...,Has Cubie killed fees?,A tale of two tails,I say what I like and I like what I say,"Elbows, Eyes and Nipples",Task force to assess risk of asteroid collision,How I found myself at last,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,McGrath puts India out of their misery,Blair Witch bandwagon rolls on,Pele turns up heat on Ferguson,Party divided over Kohl slush fund scandal,Manchester United (England),Women in record South Pole walk,Vasco da Gama (Brazil),South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,England's decade of disasters,Revenge is sweet for jubilant Cronje,"Our choice, not theirs",Profile of former US Nazi Party officer Willia...,New evidence shows record of war crimes suspec...,The rise of the supernerds,Written on the body,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
4,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,Alan Parker - part two,"Thuggery, Toxins and Ties",Met faces fresh attack on race crime,Everton fans top racist 'league of shame',"Our breasts, ourselves",Russia's new boss has an extremely strange his...,Always and forever,Most everywhere: UDIs,Most wanted: Chloe lunettes,Return of the cane 'completely off the agenda',From Sleepy Hollow to Greenel

In [135]:
import re

data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

list1 = [i for i in range(25)]
new_Index = [str(i) for i in list1]
data.columns = new_Index

for index in new_Index:
    data[index] = data[index].str.lower()
    



data.head()
#for i in range(len(data)):
#    data[i] = data[i].append(sent for sent in data[i])


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,a hindrance to operations extracts from the...,scorecard,hughes instant hit buoys blues,jack gets his skates on at ice cold alex,chaos as maracana builds up for united,depleted leicester prevail as elliott spoils e...,hungry spurs sense rich pickings,gunners so wide of an easy target,derby raise a glass to strupar s debut double,southgate strikes leeds pay the penalty,hammers hand robson a youthful lesson,saints party like it s,wear wolves have turned into lambs,stump mike catches testy gough s taunt,langer escapes to hit,flintoff injury piles on woe for england,hunters threaten jospin with new battle of the...,kohl s successor drawn into scandal,the difference between men and women,sara denver nurse turned solicitor,diana s landmine crusade put tories in a panic,yeltsin s resignation caught opposition flat f...,russian roulette,sold out,recovering a title
1,scorecard,the best lake scene,leader german sleaze inquiry,cheerio boyo,the main recommendations,has cubie killed fees,has cubie killed fees,has cubie killed fees,hopkins furious at foster s lack of hannibal...,has cubie killed fees,a tale of two tails,i say what i like and i like what i say,elbows eyes and nipples,task force to assess risk of asteroid collision,how i found myself at last,on the critical list,the timing of their lives,dear doctor,irish court halts ira man s extradition to nor...,burundi peace initiative fades after rebels re...,pe points the way forward to the ecb,campaigners keep up pressure on nazi war crime...,jane ratcliffe,yet more things you wouldn t know without the ...,millennium bug fails to bite
2,coventry caught on counter by flo,united s rivals on the road to rio,thatcher issues defence before trial by video,police help smith lay down the law at everton,tale of trautmann bears two more retellings,england on the rack,pakistan retaliate with call for video of walsh,cullinan continues his cape monopoly,mcgrath puts india out of their misery,blair witch bandwagon rolls on,pele turns up heat on ferguson,party divided over kohl slush fund scandal,manchester united england,women in record south pole walk,vasco da gama brazil,south melbourne australia,necaxa mexico,real madrid spain,raja casablanca morocco,corinthians brazil,tony s pet project,al nassr saudi arabia,ideal holmes show,pinochet leaves hospital after tests,useful links
3,pilgrim knows how to progress,thatcher facing ban,mcilroy calls for irish fighting spirit,leicester bin stadium blueprint,united braced for mexican wave,auntie back in fashion even if the dress look...,shoaib appeal goes to the top,hussain hurt by shambles but lays blame on e...,england s decade of disasters,revenge is sweet for jubilant cronje,our choice not theirs,profile of former us nazi party officer willia...,new evidence shows record of war crimes suspec...,the rise of the supernerds,written on the body,putin admits yeltsin quit to give him a head s...,bbc worst hit as digital tv begins to bite,how much can you pay for,christmas glitches,upending a table chopping a line and scoring ...,scientific evidence unreliable defence claims,fusco wins judicial review in extradition case,rebels thwart russian advance,blair orders shake up of failing nhs,lessons of law s hard heart
4,hitches and horlocks,beckham off but united survive,breast cancer screening,alan parker,guardian readers are you all whingers,hollywood beyond,ashes and diamonds,whingers a formidable minority,alan parker part two,thuggery toxins and ties,met faces fresh attack on race crime,everton fans top racist league of shame,our breasts ourselves,russia s new boss has an extremely strange his...,always and forever,most everywhere udis,most wanted chloe lunettes,return of the cane completely off the agenda,from sleepy hollow to greeneland,blunkett outlines vision for over s,embattled dobson attacks play now pay later ...,doom and the dome,what is the north south divide,aitken released from jai

In [136]:
headlines = []
for row in range(0,len(data.index)):

    headlines.append(' '.join(str(x) for x in data.iloc[row, 0:25]))

headlines[0]

#lets remove stopwords and do stemming

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

wordNet = WordNetLemmatizer()
ps = PorterStemmer()

for i in range(len(headlines)):

    if i % 500 == 0:
        print(f"Processing row {i}/{len(headlines)}")
    x = headlines[i]
    x = nltk.word_tokenize(x)
    x = [wordNet.lemmatize(word) for word in x]
    headlines[i]= ' '.join(x)

Processing row 0/3975
Processing row 500/3975
Processing row 1000/3975
Processing row 1500/3975
Processing row 2000/3975
Processing row 2500/3975
Processing row 3000/3975
Processing row 3500/3975


In [137]:
#add sentiment score
from textblob import TextBlob

sentiments = [TextBlob(text).sentiment.polarity for text in headlines]

In [138]:
#Vectorisation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy.sparse import hstack
import numpy as np

from nltk.corpus import stopwords

# 1. Get standard stopwords
stop_words = set(stopwords.words('english'))

# 2. Define important negation words
negation_words = {"not", "no", "nor", "never"}

# 3. Subtract them from the stopword list
custom_stopwords = list(stop_words - negation_words)


tf = TfidfVectorizer(stop_words=custom_stopwords, ngram_range=(2, 2), max_features=7000, min_df=2)

cv = CountVectorizer(ngram_range=(2,2))
trainDataset = tf.fit_transform(headlines)

trainDatasetCombined = hstack([trainDataset,np.array(sentiments).reshape(-1, 1)])

In [139]:
from sklearn.model_selection import GridSearchCV

from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [140]:
#Implement GridSearchCV
#After doing gridsearch
# Best parameters found from GridSearchCV

best_params_lr = {
    'C': 10,
    'class_weight': 'balanced',
    'solver': 'liblinear'
}

best_params_rf = {
    'class_weight': 'balanced',
    'max_depth': 20,
    'min_samples_leaf': 4,
    'min_samples_split': 10,
    'n_estimators': 100
}

best_params_svc = {
    'C': 100,
    'class_weight': 'balanced',
    'max_iter': 1000  # You may increase this to 3000–5000 if convergence warnings persist
}
#convergence warning for svm

if False:



    logisticRegressionModel = LogisticRegression(class_weight='balanced', max_iter=200)
    randomForest = RandomForestClassifier(n_estimators=200, criterion ='entropy', random_state=42, class_weight='balanced')
    svc = LinearSVC(class_weight='balanced', max_iter=2000)


    param_grid_lr = {
        'C': [0.01, 0.1, 1, 10],  # Regularization strength (lower = more regularization)
        'solver': ['liblinear', 'lbfgs'],  # Solver options
        'class_weight': ['balanced'],  

        }
    param_grid_rf = {
        'n_estimators': [100, 200, 300],        # Number of trees in the forest
        'max_depth': [None, 10, 20, 30],        # Max depth of the trees
        'min_samples_split': [2, 5, 10],        # Min samples to split an internal node
        'min_samples_leaf': [1, 2, 4],          # Min samples at a leaf node
        'class_weight': ['balanced'],         # Keeps your class balance
    }

    param_grid_svc = {
        'C': [0.01, 0.1, 1, 10, 100],           # Regularization strength
        'class_weight': ['balanced'],          # Account for label imbalance
        'max_iter': [1000, 2000, 3000],         # May need higher for convergence
    }

    grid_search_lr = GridSearchCV(estimator=logisticRegressionModel, param_grid=param_grid_lr, cv=5, scoring='f1_macro')
    grid_search_rf = GridSearchCV(estimator=randomForest, param_grid=param_grid_rf, cv=5, scoring='f1_macro')
    grid_search_svc = GridSearchCV(estimator=svc, param_grid=param_grid_svc, cv=5, scoring='f1_macro')

    grid_search_lr.fit(trainDataset, train_df['Label'])
    grid_search_rf.fit(trainDataset, train_df['Label'])
    grid_search_svc.fit(trainDataset, train_df['Label'])

    best_model_lr = grid_search_lr.best_estimator_
    best_model_rf = grid_search_lr.best_estimator_
    best_model_svc = grid_search_lr.best_estimator_
    print("Best parameters for lr:", grid_search_lr.best_params_)
    print("Best parameters for rf:", grid_search_rf.best_params_)
    print("Best parameters for svc:", grid_search_svc.best_params_)


In [141]:
#RandomForestClassifier
#randomForest.fit(trainDataset, train_df['Label'])
from sklearn.svm import SVC  # replaces LinearSVC

#Linear regression Model
best_model_lr = LogisticRegression(
    C=10,
    class_weight='balanced',
    solver='liblinear'
)

# Random Forest (from GridSearch)
best_model_rf = RandomForestClassifier(
    class_weight='balanced',
    max_depth=20,
    min_samples_leaf=4,
    min_samples_split=10,
    n_estimators=100,
    random_state=42
)

# Linear SVM (with higher max_iter to avoid warnings)
best_model_svc = LinearSVC(
    C=100,
    class_weight='balanced',
    max_iter=1000
)

svc_clf = SVC(
    C=100,
    class_weight='balanced',
    probability=True,     # 🔥 Key for soft voting
    max_iter=3000
)

#finalModel.fit(trainDatasetCombined, train_df['Label'])

In [142]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[
        ('lr', best_model_lr),
        ('rf', best_model_rf),
        ('svc', best_model_svc)
    ],
    voting='hard'  # Majority voting (not probability-based)
)

voting_clf.fit(trainDatasetCombined, train_df['Label'])

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=10, class_weight='balanced',
                                                 solver='liblinear')),
                             ('rf',
                              RandomForestClassifier(class_weight='balanced',
                                                     max_depth=20,
                                                     min_samples_leaf=4,
                                                     min_samples_split=10,
                                                     random_state=42)),
                             ('svc',
                              LinearSVC(C=100, class_weight='balanced'))])

In [143]:
testTransform = []

for row in range(0, len(test_df.index)):
    testTransform.append(' '.join(str(x) for x in test_df.iloc[row, 2:27]))


for i in range(len(testTransform)):

    if i % 100 == 0:
        print(f"Processing row {i}/{len(testTransform)}")
    x = re.sub( '[^a-zA-Z]', ' ',testTransform[i])
    x= x.lower()
    x = nltk.word_tokenize(x)
    x = [wordNet.lemmatize(word) for word in x]
    testTransform[i]= ' '.join(x)

sentiments2 = [TextBlob(text).sentiment.polarity for text in testTransform]



Processing row 0/126
Processing row 100/126


In [144]:
testDataset = tf.transform(testTransform)
testDatasetCombined = hstack([testDataset,np.array(sentiments2).reshape(-1, 1) ])

In [145]:
predictions = voting_clf.predict(testDatasetCombined)

In [146]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
matrix=confusion_matrix(test_df['Label'],predictions)
print(matrix)
score=accuracy_score(test_df['Label'],predictions)
print(score)
report=classification_report(test_df['Label'],predictions)
print(report)

[[31 25]
 [30 40]]
0.5634920634920635
              precision    recall  f1-score   support

           0       0.51      0.55      0.53        56
           1       0.62      0.57      0.59        70

    accuracy                           0.56       126
   macro avg       0.56      0.56      0.56       126
weighted avg       0.57      0.56      0.56       126

